In [1]:
import win32com.client
import pandas as pd
import numpy as np
import shutil
import os, sys
import openpyxl 
import time
from tqdm import tqdm
from datetime import date, timedelta, datetime
import google.cloud
from google.cloud import bigquery
import pyodbc
from pandas.io import gbq
import pandas_gbq

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"\\vspmdhq-bi\\BI_Server_F\\BI_for_Reservations\\GCP Python credentials\\nh-gem.json"

def preparar_gbq(dataset):
    Call = dataset.astype(str)
    dictionary = {} 
    [dictionary.update({f'{i}':str}) for i in dataset.columns]
    Call_db = Call.astype(dictionary)
    return Call_db

In [2]:
print('Actualizando Creators')
creators = pd.read_excel(r'\\Spmdct-nas\GCC\DB\Operational Reports\BigQuery - R\Scripts\Python\Maestros_Diarios\CREATORS.xlsx').dropna()
creators = creators.reset_index().iloc[:,1:]
creators.columns = ['Code', 'Name']
creators.Code = creators.Code.astype(float).astype(int).astype(str).str.zfill(10)
Call_db = preparar_gbq(creators)
client = bigquery.Client(project='nh-gem')
table = client.dataset("Masters").table("Creators")
job_config = bigquery.job.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
bigqueryJob = client.load_table_from_dataframe(Call_db, table, job_config=job_config)
bigqueryJob.result()
pandas_gbq.to_gbq(Call_db, destination_table='Masters.Creators',project_id='nh-gem', if_exists='replace')

print('Creators Actualizado')

print('Actualizando AH AGENCY')
Account_Handlers = pd.read_excel(r'\\Spmdct-nas\GCC\DB\Operational Reports\BigQuery - R\Scripts\Python\Maestros_Diarios\BID_AH_AGENCIES.xlsx', engine='openpyxl')
Account_Handlers.columns =  ['BF', 'Reservation_id', 'AH']
Call_db = preparar_gbq(Account_Handlers)
client = bigquery.Client(project='nh-gem')
table = client.dataset("Masters").table("Account_Handlers_Agencies")
job_config = bigquery.job.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
bigqueryJob = client.load_table_from_dataframe(Call_db, table, job_config=job_config)
bigqueryJob.result()

print('AH Agency Actualizado')


print('CONECTANDO AL SQL SERVER') 
conn = pyodbc.connect("Driver={SQL Server};"
            "Server=VSPMDHQ-BI2;"
            "Database=Meetings;"
            "Trusted_Connection=yes;")
cursor = conn.cursor()

print('Conectado a SQL SERVER')

print('Sacamos BOMT_Inputs')
colnames2 = 'BF_ID, BF_GroupName, Observation_Day, Obs_Key, TRev_F_EUR, RS_ID, BF_Creator, BF_Creator_Center, BF_2Agent, xBF_Confirmation_Date, BF_Channel, xBF_MainCustomer, xBF_MainCustomer_Branch, BF_FollowUp, xBF_CRS, xBF_CRS_Branch, BF_ContactPerson, BF_ContactTelephone, BF_ContactEmail, BF_Remarks, BF_Arrival_Date, BF_Size, xBF_Status, RS_Status'
colnames1 = 'BF_ID, BF_GroupName, Observation_Day, Obs_Key, sum(TRev_F_EUR) as TRev_F_EUR, RS_ID, BF_Creator, BF_Creator_Center, BF_2Agent, xBF_Confirmation_Date, BF_Channel, xBF_MainCustomer, xBF_MainCustomer_Branch, BF_FollowUp, xBF_CRS, xBF_CRS_Branch, BF_ContactPerson, BF_ContactTelephone, BF_ContactEmail, BF_Remarks, BF_Arrival_Date, BF_Size, xBF_Status, RS_Status'
colnames = 'BF_ID, BF_GroupName, Observation_Day, Obs_Key, RS_ID, BF_Creator, BF_Creator_Center, BF_2Agent, xBF_Confirmation_Date, BF_Channel, xBF_MainCustomer, xBF_MainCustomer_Branch, BF_FollowUp, xBF_CRS, xBF_CRS_Branch, BF_ContactPerson, BF_ContactTelephone, BF_ContactEmail, BF_Remarks, BF_Arrival_Date, BF_Size, xBF_Status, RS_Status'
sql = f"select {colnames1} from BOMT_Inputs WHERE Obs_Key='TODAY' GROUP BY {colnames}"
cursor.execute(sql)
colnames = [column[0] for column in cursor.description]
raw_data = cursor.fetchall()
BOMT_Inputs = pd.DataFrame(np.array(list(raw_data)), columns=colnames)
names = BOMT_Inputs.columns.tolist()
names[names.index('BF_ID')] = 'BF'
BOMT_Inputs.columns = names

Call_db = preparar_gbq(BOMT_Inputs)
pandas_gbq.to_gbq(Call_db, destination_table='Masters.BOMT_Inputs',project_id='nh-gem', if_exists='replace')

print('BOMT_Inputs Actualizado')

Actualizando Creators


c:\Users\ej.garcia\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
100%|██████████| 1/1 [00:00<00:00, 174.62it/s]


Creators Actualizado
Actualizando AH AGENCY


c:\Users\ej.garcia\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


AH Agency Actualizado
CONECTANDO AL SQL SERVER
Conectado a SQL SERVER
Sacamos BOMT_Inputs


100%|██████████| 1/1 [00:00<?, ?it/s]

BOMT_Inputs Actualizado
